## Different models we tried for the competition

Most of GridSearches and attempts are in the notebook named PIU-model-Copy1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
train = pd.read_csv("../data/cleaned_train.csv")
test = pd.read_csv("../data/test.csv")
pd.set_option('display.max_columns', None)
train

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,complete_resp_total
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.061430,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0,2.0,55.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.211720,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,453.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,0.0,28.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,577.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.220330,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0,1.0,44.0
4,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,NaN,60.0,73.0,102.0,NaN,NaN,NaN,Summer,12.0,0.0,16.5,2.0,17.9,2.0,6.0,0.0,10.0,1.0,11.0,1.0,8.0,0.0,Summer,2.0,4.33036,30.1865,1330.970,1996.45,30.21240,84.0285,16.6877,13.498800,67.97150,2.0,32.9141,20.90200,79.6982,35.3804,63.1265,NaN,NaN,Spring,4.110,Summer,40.0,56.0,Spring,0.0,1.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,ff6c2bb8,Fall,8,0,NaN,NaN,Fall,17.139810,52.5,67.2,25.0,60.0,65.0,112.0,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,8.0,1.0,10.0,1.0,12.0,1.0,Fall,3.0,3.20303,17.1417,1035.270,1759.96,11.00630,52.5331,13.4004,3.741300,14.66690,1.0,25.7118,15.81500,49.3301,20.2645,36.7181,NaN,NaN,Fall,3.440,Fall,41.0,58.0,Fall,2.0,0.0,22.0
2667,ff759544,Summer,7,1,NaN,NaN,Summer,13.927006,48.5,46.6,23.0,65.0,75.0,105.0,NaN,NaN,NaN,Summer,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,9.0,0.0,8.5,0.0,4.5,0.0,Fall,1.0,2.36680,13.6457,966.287,1256.17,9.98802,45.1853,13.2315,0.414263,1.41470,1.0,20.0572,15.14000,42.8185,18.0937,30.0453,NaN,NaN,NaN,NaN,Summer,48.0,67.0,Summer,0.0,1.0,33.0
2668,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,NaN,71.0,70.0,104.0,NaN,NaN,NaN,Fall,16.0,0.0,18.0,1.0,19.9,2.0,10.0,1.0,8.0,1.0,9.0,1.0,12.0,1.0,Fall,3.0,4.52277,16.3642,1206.880,2051.70,19.46110,70.8117,14.0629,2.301380,11.58830,1.0,33.3709,17.97970,66.2889,29.7790,52.8320,NaN,NaN,Winter,3.260,Winter,35.0,50.0,Fall,1.0,1.0,32.0
2669,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,NaN,79.0,99.0,116.0,NaN,NaN,NaN,Winter,15.0,1.0,18.5,2.0,15.8,2.0,0.0,0.0,10.0,1.0,10.0,1.0,

In [3]:
# Identify columns to encode (all that end with '-Season')
columns_to_encode = [col for col in train.columns if col.endswith('Season')]

# Perform one-hot encoding
train = pd.get_dummies(train, columns=columns_to_encode, prefix=columns_to_encode)
test = pd.get_dummies(test, columns=columns_to_encode, prefix=columns_to_encode)

In [4]:
test['is_test'] = True  # Add a flag to identify test data
train['is_test'] = False
combined_data = pd.concat([train, test], ignore_index=True)

In [5]:
# Split back into train and test datasets
train_data_preprocessed = combined_data[combined_data['is_test'] == False].drop(columns=['is_test'])
test_data_preprocessed = combined_data[combined_data['is_test'] == True].drop(columns=['is_test'])
train_data_preprocessed = train_data_preprocessed.drop(columns="id")
# Print the results
print("Preprocessed Train Data:")
print(train_data_preprocessed)

print("\nPreprocessed Test Data:")
print(test_data_preprocessed)

X = train_data_preprocessed.drop(columns=['sii', 'complete_resp_total'])
y = train_data_preprocessed['complete_resp_total']

Preprocessed Train Data:
      Basic_Demos-Age  Basic_Demos-Sex  CGAS-CGAS_Score  Physical-BMI  \
0                   5                0             51.0     16.877316   
1                   9                0              NaN     14.035590   
2                  10                1             71.0     16.648696   
3                   9                0             71.0     18.292347   
4                  13                1             50.0     22.279952   
...               ...              ...              ...           ...   
2666                8                0              NaN     17.139810   
2667                7                1              NaN     13.927006   
2668               13                0             60.0     16.362460   
2669               11                0             68.0     21.441500   
2670               13                0             70.0     12.235895   

      Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
0                46.0     

In [6]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

categorical_features = [
    'Basic_Demos-Sex',
    'FGC-FGC_CU_Zone',
    'FGC-FGC_GSND_Zone',
    'FGC-FGC_GSD_Zone',
    'FGC-FGC_PU_Zone',
    'FGC-FGC_SRL_Zone',
    'FGC-FGC_SRR_Zone',
    'FGC-FGC_TL_Zone',
    'BIA-BIA_Activity_Level_num',
    'BIA-BIA_Frame_num',
    'PreInt_EduHx-computerinternet_hoursday'
]
X[categorical_features] = X[categorical_features].astype(str)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## CatBoost before GridSearch

In [7]:
# Create CatBoost Pool for better handling of categorical features
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features)
test_pool = Pool(data=X_val, label=y_val, cat_features=categorical_features)

# Initialize CatBoostRegressor
model = CatBoostRegressor(
    iterations=500,       # Number of boosting iterations
    learning_rate=0.1,    # Step size for weight updates
    depth=6,              # Depth of the trees
    loss_function='RMSE', # Regression problem
    verbose=50            # Print training progress every 50 iterations
)

# Train the model
model.fit(train_pool)

# Predict on the test set
y_pred = model.predict(test_pool)

# Evaluate the model
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"Root Mean Squared Error: {rmse}")

0:	learn: 19.7911918	total: 175ms	remaining: 1m 27s
50:	learn: 15.5394596	total: 1.83s	remaining: 16.2s
100:	learn: 14.6263336	total: 3.45s	remaining: 13.6s
150:	learn: 13.7622483	total: 5.56s	remaining: 12.8s
200:	learn: 12.7666285	total: 7.84s	remaining: 11.7s
250:	learn: 11.9357267	total: 9.54s	remaining: 9.46s
300:	learn: 11.0896860	total: 11.1s	remaining: 7.33s
350:	learn: 10.4996128	total: 12.6s	remaining: 5.36s
400:	learn: 9.8502483	total: 14.1s	remaining: 3.49s
450:	learn: 9.2946670	total: 15.6s	remaining: 1.7s
499:	learn: 8.7744558	total: 17.3s	remaining: 0us
Root Mean Squared Error: 17.444624174594342


C:\Users\uku1\.conda\envs\IDS\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## CatBoost after GridSearch

In [8]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(
    iterations=1000,       # Number of boosting iterations
    learning_rate=0.01,    # Step size for weight updates
    depth=8,              # Depth of the trees
    loss_function='RMSE', # Regression problem
    verbose=200,
    cat_features=categorical_features,
    l2_leaf_reg=5)

# Define scoring metric (neg_mean_squared_error for regression)
scoring = 'neg_mean_squared_error'

# Perform Cross-Validation
scores = cross_val_score(model, X, y, cv=5, scoring=scoring)

# Convert to positive MSE
mse_scores = -scores

# Calculate RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

# Print Results
print("Mean RMSE:", mean_rmse)
print("RMSE Scores for each fold:", rmse_scores_per_fold)

0:	learn: 20.3763306	total: 22.8ms	remaining: 22.8s
200:	learn: 16.3526096	total: 12.1s	remaining: 48.2s
400:	learn: 15.0544507	total: 24.2s	remaining: 36.1s
600:	learn: 14.2696249	total: 37.4s	remaining: 24.8s
800:	learn: 13.6697139	total: 50.2s	remaining: 12.5s
999:	learn: 13.2618908	total: 1m 3s	remaining: 0us
0:	learn: 20.2888132	total: 66.7ms	remaining: 1m 6s
200:	learn: 16.3064829	total: 11.2s	remaining: 44.6s
400:	learn: 15.1098822	total: 24.2s	remaining: 36.1s
600:	learn: 14.3396827	total: 36.6s	remaining: 24.3s
800:	learn: 13.7702661	total: 47s	remaining: 11.7s
999:	learn: 13.2187855	total: 1m	remaining: 0us
0:	learn: 20.2624518	total: 64.8ms	remaining: 1m 4s
200:	learn: 16.2868692	total: 12.5s	remaining: 49.6s
400:	learn: 15.0495072	total: 25.1s	remaining: 37.4s
600:	learn: 14.3750295	total: 37.1s	remaining: 24.6s
800:	learn: 13.8187967	total: 48.8s	remaining: 12.1s
999:	learn: 13.3866267	total: 59.9s	remaining: 0us
0:	learn: 19.9212941	total: 43.2ms	remaining: 43.1s
200:	lea

## Random Forest

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Create a OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical_features),
    ],
    remainder='passthrough'  # This will keep numerical columns as they are if not scaled.
)

rf_model = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('rf', RandomForestRegressor(
        n_estimators=50,    # Number of trees
        max_depth=5,         # Similar depth to CatBoost example
        min_samples_split=2,
        min_samples_leaf=2,
        random_state=42      # For reproducibility
    ))
])

# Define scoring metric (neg_mean_squared_error for regression)
scoring = 'neg_mean_squared_error'

# Perform Cross-Validation
scores = cross_val_score(rf_model, X, y, cv=5, scoring=scoring)

# Convert to positive MSE
mse_scores = -scores

# Calculate RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

# Print Results
print("Mean RMSE:", mean_rmse)
print("RMSE Scores for each fold:", rmse_scores_per_fold)

Mean RMSE: 17.22168221698686
RMSE Scores for each fold: [17.22430661 16.85621766 16.57147982 17.97996605 17.44218328]


## Ensemble of RF and CatBoost

In [10]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# You already have:
# model: a CatBoostRegressor instance with specified parameters (not yet fitted)
# rf_model: a pipeline that encodes categorical features and applies RandomForestRegressor

# Create an ensemble of the CatBoost model and the Random Forest model
ensemble = VotingRegressor(estimators=[
    ('cat', model),
    ('rf', rf_model)
])

# Perform cross-validation on the ensemble
scoring = 'neg_mean_squared_error'
scores = cross_val_score(ensemble, X, y, cv=5, scoring=scoring)

# Convert from negative MSE to positive MSE
mse_scores = -scores

# Compute RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

print("Mean RMSE (Ensemble):", mean_rmse)
print("RMSE Scores for each fold (Ensemble):", rmse_scores_per_fold)

0:	learn: 20.3763306	total: 19.8ms	remaining: 19.8s
200:	learn: 16.3526096	total: 9.83s	remaining: 39.1s
400:	learn: 15.0544507	total: 23.4s	remaining: 35s
600:	learn: 14.2696249	total: 34.5s	remaining: 22.9s
800:	learn: 13.6697139	total: 47.4s	remaining: 11.8s
999:	learn: 13.2618908	total: 57.7s	remaining: 0us
0:	learn: 20.2888132	total: 47.9ms	remaining: 47.9s
200:	learn: 16.3064829	total: 12.7s	remaining: 50.7s
400:	learn: 15.1098822	total: 24.1s	remaining: 35.9s
600:	learn: 14.3396827	total: 35.8s	remaining: 23.8s
800:	learn: 13.7702661	total: 47.8s	remaining: 11.9s
999:	learn: 13.2187855	total: 1m	remaining: 0us
0:	learn: 20.2624518	total: 81.3ms	remaining: 1m 21s
200:	learn: 16.2868692	total: 11.9s	remaining: 47.1s
400:	learn: 15.0495072	total: 24.3s	remaining: 36.2s
600:	learn: 14.3750295	total: 36.9s	remaining: 24.5s
800:	learn: 13.8187967	total: 47.3s	remaining: 11.8s
999:	learn: 13.3866267	total: 59.8s	remaining: 0us
0:	learn: 19.9212941	total: 64.8ms	remaining: 1m 4s
200:	le

## XGBoost

In [11]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV

# Convert object columns to categorical or numeric
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')

# Define the parameter grid
param_grid = {
    #'n_estimators': [100, 500, 1000],        # Number of boosting rounds
    #'learning_rate': [0.01, 0.1, 0.2],      # Step size
    #'max_depth': [6, 8, 10],                # Tree depth
    #'reg_lambda': [1, 5, 10],               # L2 regularization
    #'min_child_weight': [1, 3, 5],          # Minimum sum of instance weights for child
    #'subsample': [0.8, 1.0],                # Subsample ratio of the training data
    #'colsample_bytree': [0.8, 1.0]          # Subsample ratio of columns
}

# Initialize the XGBRegressor
model = XGBRegressor(objective='reg:squarederror', enable_categorical=True)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error', # Scoring metric
    cv=5,                             # Number of cross-validation folds
    verbose=2,                        # Verbosity level
    n_jobs=-1                         # Use all available cores
)

# Perform the grid search
grid_search.fit(X, y)

# Get the best parameters and score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert to positive MSE

# Calculate RMSE from the best score
best_rmse = np.sqrt(best_score)

print("Best Parameters:", best_params)
print("Best RMSE:", best_rmse)

'''
# Define scoring metric (neg_mean_squared_error for regression)
scoring = 'neg_mean_squared_error'

# Perform Cross-Validation
scores = cross_val_score(model, X, y, cv=5, scoring=scoring)

# Convert to positive MSE
mse_scores = -scores

# Calculate RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

# Print Results
print("Mean RMSE:", mean_rmse)
print("RMSE Scores for each fold:", rmse_scores_per_fold)
'''

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters: {}
Best RMSE: 18.675698150405868


'\n# Define scoring metric (neg_mean_squared_error for regression)\nscoring = \'neg_mean_squared_error\'\n\n# Perform Cross-Validation\nscores = cross_val_score(model, X, y, cv=5, scoring=scoring)\n\n# Convert to positive MSE\nmse_scores = -scores\n\n# Calculate RMSE\nmean_rmse = np.sqrt(np.mean(mse_scores))\nrmse_scores_per_fold = np.sqrt(mse_scores)\n\n# Print Results\nprint("Mean RMSE:", mean_rmse)\nprint("RMSE Scores for each fold:", rmse_scores_per_fold)\n'

In [12]:
# Initialize XGBRegressor with categorical support enabled
model = XGBRegressor(
    n_estimators=500,          # Number of boosting rounds
    learning_rate=0.01,         # Step size for weight updates
    max_depth=8,                # Maximum depth of trees
    objective='reg:squarederror', # Regression with RMSE as loss function
    verbosity=1,                # Logging level
    reg_lambda=3,               # L2 regularization term
    enable_categorical=True     # Enable categorical feature support
)

# Define scoring metric (neg_mean_squared_error for regression)
scoring = 'neg_mean_squared_error'

# Perform Cross-Validation
scores = cross_val_score(model, X, y, cv=5, scoring=scoring)

# Convert to positive MSE
mse_scores = -scores

# Calculate RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

# Print Results
print("Mean RMSE:", mean_rmse)
print("RMSE Scores for each fold:", rmse_scores_per_fold)

Mean RMSE: 17.42924896539261
RMSE Scores for each fold: [17.30934468 17.00026335 16.76964611 18.06621353 17.96279846]
